#Árbol de decisión

Facultad de Ingeniería 

Maestría en Inteligencia Artificial 

Machine Learning 

Cecilia Gabriela Rodríguez Flores 

Sheila Leyva López 

## Librerias

In [ ]:
import pandas as pd
import math
import numpy as np
import operator
from pandas.core.algorithms import unique

## Funciones

### Funciones para generar el arbol

In [ ]:
def contar_clases(atributo_decision, clases):
    cont_clases = np.zeros((len(clases)))
    for n, c in enumerate(clases):
        cont_clases[n] = int(np.sum(atributo_decision == c))
    return cont_clases

In [ ]:
def calcular_entropia(cont_clases):
    entropia_decision = 0
    total = int(sum(cont_clases))
    for d in cont_clases:
        if d == 0: entropia_decision += 0
        else: entropia_decision -= (d/total)*np.log2(d/total)
    return entropia_decision

In [ ]:
def ganancia_atributo(x, y, cont_clases, entropia, clases):
    Total = int(sum(cont_clases))
    ganancia = {}
    for a in x.columns:
        ganancia[a] = 0
        observaciones = x[a].unique()
        for o in observaciones:
            y_a_o = y.loc[x[a] == o]
            cont_clases_y_a_o = contar_clases(y_a_o, clases)
            entropia_a_o = calcular_entropia(cont_clases_y_a_o)
            total = int(sum(cont_clases_y_a_o))
            ganancia[a] += (total/Total) * entropia_a_o
        ganancia[a] = entropia - ganancia[a]
    return ganancia

In [ ]:
def nodo_raiz(ganancia):
    raiz = max(ganancia.items(), key=operator.itemgetter(1))[0]
    entropia_r = ganancia[raiz]
    return raiz, entropia_r

In [ ]:
def nodos_intermedios(x, y, atributo, obs):
    subx = []
    suby = []
    for o in obs:
        pos_coinciden = x[atributo] == o
        subx.append(x.loc[pos_coinciden])
        suby.append(y.loc[pos_coinciden])
    return subx, suby

In [ ]:
def rama_final(y_fr, obs):
    hoja = 0
    clase_fr = 0
    n = len(y_fr)
    for o in obs:
        coincidencias = int(np.sum(y_fr == o))
        if coincidencias == n:
            hoja = 1
            clase_fr = o
    return hoja, clase_fr

In [ ]:
def arbol_id3(x, y, clases, arbol):
    conteo_clases = contar_clases(y, clases)
    entropia_decision = calcular_entropia(conteo_clases)
    ganancia = ganancia_atributo(x, y, conteo_clases, entropia_decision, clases)
    nodo_raiz_, entropia_raiz = nodo_raiz(ganancia)
    arbol[nodo_raiz_] = {}
    ramas = x[nodo_raiz_].unique()
    sub_x, sub_y = nodos_intermedios(x, y, nodo_raiz_, ramas)
    print(sub_y)
    print(clases)
    print('')
    for r, sx, sy in zip(ramas, sub_x, sub_y):
        arbol[nodo_raiz_][r] = {}
        hoja, clase_fr = rama_final(sy, clases)
        if hoja == 1:
            arbol[nodo_raiz_][r] = clase_fr
        else:
            arbol_id3(sx, sy, clases, arbol[nodo_raiz_][r])
    return arbol

### Función para separa la base de datos en entrenamiento y prueba

In [ ]:
def division_80_20(datos, etiquetas):
  n = len(datos)
  indices = np.arange(n)
  np.random.shuffle(indices)
  lim = int(n * 0.80) + 1
  x_train = datos.loc[indices[0:lim]]
  x_test = datos.loc[indices[lim:]]
  y_train = etiquetas.loc[indices[0:lim]]
  y_test = etiquetas.loc[indices[lim:]]
  return x_train, x_test, y_train, y_test

### Funciones evaluar algoritmo

In [ ]:
def exactitud(y_calculada, y_real):
  coincidencias = np.equal(y_calculada, y_real)
  total_coincidencias = np.sum(coincidencias)
  porcentaje = (total_coincidencias / len(y_calculada))
  return porcentaje

In [ ]:
def precision(y_class, y_real):
    TP = FP = 0
    for c, r in zip(y_class, y_real):
        if r == 'Setosa':
            if ((r == c) and r == 'Setosa'): TP += 1
            elif ((r != c) and ( r == 'Virginica' or r == 'Versicolor')): FP += 1
        elif r == 'Virginica':
            if ((r == c) and r == 'Virginica'): TP += 1
            elif ((r != c) and ( r == 'Setosa' or r == 'Versicolor')): FP += 1
        elif r == 'Versicolor':
            if ((r == c) and r == 'Versicolor'): TP += 1
            elif ((r != c) and ( r == 'Virginica' or r == 'Setosa')): FP += 1
    print('TP:',TP)
    print('FP:',FP)
    Pr = TP / (TP + FP)
    return Pr

## Base de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Lectura de la base de datos

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Machine Learning/iris.csv')
columnas = df.columns.values

In [ ]:
df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Separación de los datos y etiquetas

In [ ]:
x = df[columnas[:4]].copy()
y = df['variety']
clases = y.unique()

In [ ]:
x

In [ ]:
clases

Comprobación de la falta de valores faltantes

In [ ]:
df.isnull().sum()

Observaciones de los atributos 

In [ ]:
elem_columnas = {}
for c in columnas:
  elem_columnas[c] = df[c].unique()
elem_columnas

Conjunto de entrenamiento y prueba

In [ ]:
x_train, x_test, y_train, y_test = division_80_20(x, y)

## Etapa de Entrenamiento

In [ ]:
arbol_iris = {}
arbol_iris = arbol_id3(x_train, y_train, clases, arbol_iris)

In [ ]:
arbol_iris

### Función comprobación

In [ ]:
def comprobacion(x_clasificar):
    petal_l = x_clasificar['petal.length']
    petal_w = x_clasificar['petal.width']
    sepal_l = x_clasificar['sepal.length']
    sepal_w = x_clasificar['sepal.width']

    resultado = 'No se econtró'
   # print(type(petal_l))
   # print(sepal_l)

    if petal_l == 5.1:
        if sepal_l == 5.8:
            resultado = 'Virginica'
        elif sepal_l == 6.9:
            resultado = 'Virginica'
        elif sepal_l == 5.9:
            resultado = 'Virginica'
        elif sepal_l == 6.0:
            resultado = 'Versicolor'
        elif sepal_l == 6.5:
            resultado = 'Virginica'

    elif petal_l == 6.1:
        resultado = 'Virginica'
    elif petal_l == 1.7:
        resultado = 'Setosa'
    elif petal_l == 5.6:
        resultado = 'Virginica'
    elif petal_l == 4.8:
        if sepal_l == 6.8:
            resultado = 'Versicolor'
        elif sepal_l == 6.2:
            resultado = 'Virginica'
        elif sepal_l == 6.0:
            resultado = 'Virginica'
        elif sepal_l == 4.9:
            resultado = 'Versicolor'

    elif petal_l == 5.7:
        resultado = 'Virginica'
    elif petal_l == 3.7:
        resultado = 'Versicolor'
    elif petal_l == 5.8:
        resultado = 'Virginica'
    elif petal_l == 1.4:
        resultado = 'Setosa'
    elif petal_l == 6.6:
        resultado = 'Virginica'


    elif petal_l == 4.4:
        resultado = 'Versicolor'
    elif petal_l == 4.6:
        resultado = 'Versicolor'
    elif petal_l == 5.9:
        resultado = 'Virginica'
    elif petal_l == 1.5:
        resultado = 'Setosa'
    elif petal_l == 3.9:
        resultado = 'Versicolor'
    elif petal_l == 5.0:
        resultado = 'Virginica'
    elif petal_l == 6.0:
        resultado = 'Virginica'
    elif petal_l == 4.2:
        resultado = 'Versicolor'
    elif petal_l == 4.5:
        resultado = 'Versicolor'
    elif petal_l == 6.4:
        resultado = 'Virginica'
    elif petal_l == 1.3:
        resultado = 'Setosa'
    elif petal_l == 5.3:
        resultado = 'Virginica'
    elif petal_l == 3.0:
        resultado = 'Versicolor'
    elif petal_l == 6.3:
        resultado = 'Virginica'
    elif petal_l == 5.2:
        resultado = 'Virginica'
    elif petal_l == 5.5:
        resultado = 'Virginica'
    elif petal_l == 4.9:

        if sepal_w == 2.5:
            resultado = 'Versicolor'
        elif sepal_w == 2.7:
            resultado = 'Virginica'
        elif sepal_w == 3.1:
            resultado = 'Versicolor'   

    elif petal_l == 4.7:
        resultado = 'Versicolor'
    elif petal_l == 4.0:
        resultado = 'Versicolor'
    elif petal_l == 3.6:
        resultado = 'Versicolor'
    elif petal_l == 1.9:
        resultado = 'Setosa'
    elif petal_l == 1.0:
        resultado = 'Setosa'
    elif petal_l == 1.6:
        resultado = 'Setosa'

    elif petal_l == 6.4:
        resultado = 'Virginica'
    elif petal_l == 3.5:
        resultado = 'Versicolor'
    elif petal_l == 3.8:
        resultado = 'Versicolor'

    elif petal_l == 6.9:
        resultado = 'Virginica'
    elif petal_l == 4.1:
        resultado = 'Versicolor'
    elif petal_l == 4.3:
        resultado = 'Versicolor'
    elif petal_l == 3.3:
        resultado = 'Versicolor'

    elif petal_l == 1.1:
        resultado = 'Setosa'
    elif petal_l == 5.4:
        resultado = 'Virginica'
   # print(resultado)
    return resultado

In [ ]:
def clasificar_id3(x):
  indices = x.index
  clasificaciones = []
  for i in indices:
    clasifica = x.loc[i]
    resultado_i = comprobacion(clasifica)
    clasificaciones.append(resultado_i)
  return clasificaciones

## Evaluación del árbol

Etapa de entrenamiento

In [ ]:
y_clasificada_train = clasificar_id3(x_train)

In [ ]:
coincidencias = np.equal(y_clasificada_train, y_train)

In [ ]:
exactitud_train = exactitud(y_clasificada_train, y_train)
precision_train = precision(y_clasificada_train, y_train)

In [ ]:
print(f'Exactitud: {exactitud_train}')

In [ ]:
print(f'Precisión: {precision_train}')

Etapa de prueba

In [ ]:
y_clasificada = clasificar_id3(x_test)

In [ ]:
exactitud_test = exactitud(y_clasificada, y_test)
precision_test = precision(y_clasificada, y_test)

In [ ]:
print(f'Exactitud: {exactitud_test}')

In [ ]:
print(f'Precisión: {precision_test}')